In [ ]:
import os
import csv
import numpy as np 
import pandas as pd 


with open("ecommerce_sales.csv", "r", encoding="utf-8") as file:
    reader = csv.reader(file)  

df = pd.read_csv("ecommerce_sales.csv")  


print("Dublettrader:", df.duplicated().sum()) # undersöker om det finns några dublettrader i filen
print(df.isna().sum()) #undersöker om det finns några Null-värden i filen

In [ ]:
# tar ut försäljning per stad, högst försäljning i topp

revenue_per_city = df.groupby("city")["revenue"].sum()
revenue_per_city = revenue_per_city.sort_values(ascending=False)

print(revenue_per_city)

In [ ]:
# tar ut de 3 produkterna med högst försäljningsvärde

top_3 = df.groupby("category")["revenue"].sum()
top_3 = top_3.sort_values(ascending=False).head(3)
print(top_3)